In [1]:
import numpy
import pandas as pd
import seaborn
import nltk
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow import keras


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
dataframe = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/restaurant_reviews.csv');

In [4]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Restaurant  10000 non-null  object 
 1   Reviewer    9962 non-null   object 
 2   Review      9955 non-null   object 
 3   Rating      9962 non-null   float64
 4   Metadata    9962 non-null   object 
 5   Time        9962 non-null   object 
 6   Pictures    10000 non-null  int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 547.0+ KB


In [5]:
dataframe.drop(columns=['Restaurant', 'Reviewer', 'Metadata','Time','Pictures'], inplace=True)
dataframe = dataframe.dropna(how='any',axis=0)

In [6]:
 import nltk
 nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import string
def delete_stop_word(text):       
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result 

In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
dataframe['Review']

0       The ambience was good, food was quite good . h...
1       Ambience is too good for a pleasant evening. S...
2       A must try.. great food great ambience. Thnx f...
3       Soumen das and Arun was a great guy. Only beca...
4       Food is good.we ordered Kodi drumsticks and ba...
                              ...                        
9995    Madhumathi Mahajan Well to start with nice cou...
9996    This place has never disappointed us.. The foo...
9997    Bad rating is mainly because of "Chicken Bone ...
9998    I personally love and prefer Chinese Food. Had...
9999    Checked in here to try some delicious chinese ...
Name: Review, Length: 9955, dtype: object

In [10]:
dataframe['Review'] = str(dataframe['Review'])
dataframe['Clean'] = dataframe['Review'].apply(delete_stop_word)


In [11]:
dataframe['Clean']

0       [ambience, good, food, good, ambience, good, p...
1       [ambience, good, food, good, ambience, good, p...
2       [ambience, good, food, good, ambience, good, p...
3       [ambience, good, food, good, ambience, good, p...
4       [ambience, good, food, good, ambience, good, p...
                              ...                        
9995    [ambience, good, food, good, ambience, good, p...
9996    [ambience, good, food, good, ambience, good, p...
9997    [ambience, good, food, good, ambience, good, p...
9998    [ambience, good, food, good, ambience, good, p...
9999    [ambience, good, food, good, ambience, good, p...
Name: Clean, Length: 9955, dtype: object

In [12]:
from nltk.tokenize import word_tokenize
dataframe['Clean'] = str(dataframe['Clean'])
dataframe['Clean'] = dataframe['Clean'].apply(word_tokenize)

In [13]:
word_tokenizer = []
for sent in dataframe['Clean'] :
    sent = str(sent)
    word_tokenizer.append(word_tokenize(sent))


In [14]:
dataframe.drop(columns=['Review'], inplace = True)


In [15]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [16]:
def stem_words(text):       
    return " ".join([stemmer.stem(word) for word in text])
dataframe['Clean'] = dataframe['Clean'].apply(lambda text: stem_words(text))

In [17]:
dataframe['Clean']

0       0 [ ambienc , good , food , good , ambienc , g...
1       0 [ ambienc , good , food , good , ambienc , g...
2       0 [ ambienc , good , food , good , ambienc , g...
3       0 [ ambienc , good , food , good , ambienc , g...
4       0 [ ambienc , good , food , good , ambienc , g...
                              ...                        
9995    0 [ ambienc , good , food , good , ambienc , g...
9996    0 [ ambienc , good , food , good , ambienc , g...
9997    0 [ ambienc , good , food , good , ambienc , g...
9998    0 [ ambienc , good , food , good , ambienc , g...
9999    0 [ ambienc , good , food , good , ambienc , g...
Name: Clean, Length: 9955, dtype: object

In [27]:
dataframe['Rating'][2362]


5.0

In [19]:
dataframe = dataframe[['Clean','Rating']]
dataframe.dropna()
dataframe['Rating'][7601] = 5.0

In [20]:
dataframe['Rating']  = dataframe['Rating'].astype('double')

In [21]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9955 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Clean   9955 non-null   object 
 1   Rating  9955 non-null   float64
dtypes: float64(1), object(1)
memory usage: 553.3+ KB


In [22]:
dataframe['Rating'][2359:-1]

2359    5.0
2361    4.0
2362    5.0
2363    5.0
2364    5.0
       ... 
9994    1.5
9995    3.0
9996    4.5
9997    1.5
9998    4.0
Name: Rating, Length: 7595, dtype: float64

In [23]:
dataframe['Rating'][2360] = 5.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 10000)
X = cv.fit_transform(dataframe['Clean']).toarray()


In [25]:
y = []
for i in range(0,9955):
    if(dataframe['Rating'][i] >=1.0 and dataframe['Rating'][i] <2.0):
        y.append([1,0,0,0,0])
    elif(dataframe['Rating'][i] >=2.0 and dataframe['Rating'][i] <3.0):
        y.append([0,1,0,0,0])
    elif(dataframe['Rating'][i] >=3.0 and dataframe['Rating'][i] <4.0):
        y.append([0,0,1,0,0])
    elif(dataframe['Rating'][i] >=4.0 and dataframe['Rating'][i] <5.0):
        y.append([0,0,0,1,0])
    elif(dataframe['Rating'][i] == 5.0):
        y.append([0,0,0,0,1])
    else:
        y.append([0,0,0,0,0])
    

KeyError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.0, random_state = 0)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:

y_train[3]

In [ ]:
model = keras.Sequential([
   
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100)